In [11]:
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc
!pip install tempfile2

ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mdbtools is already the newest version (1.0.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
  Preparing metadata (setup.py) ... done
  Created wheel for tempfile2: filename=tempfile2-0.1.2-py3-none-any.whl size=2713 sha256=24587955e3c9a6eb3403fff406ea2d1a2fe87ccd79feb74bda6d296dc95c69cc
  Stored in directory: /root/.cache/pip/wheels/d8/13/c7/7053b5958dafad5cb95b4e26b0df0dcf6ae3a7aa03fe1a02f9
Successfully built tempfile2


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')
import numpy as np

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util
from IPython.display import display
from tqdm import trange

In [23]:

# 使用例
table_name = 'HorseResult'  # テーブル名を指定

file_list = [f'kjdb_prediction_cache_{year}.accdb' for year in range(2000, 2025)]
years = range(2000, 2025)

k_a_util.prepare_pd_on_notebook()
print(len(file_list))

dict_statistics={}
for i in trange(len(years)):
  y = years[i]
  filepath = f'/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_{y}.accdb'
  df_summary = k_a_util.read_table(filepath, table_name)
  columns_to_drop = [ 'rr_r_id','rr_r_horse_id','rr_r_horse_no', 'upd']
  df = df_summary.drop(columns=columns_to_drop)
  dict_statistics[y] = df

25


100%|██████████| 25/25 [03:07<00:00,  7.50s/it]


In [24]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_histograms(dict_statistics, rows, cols, dpi=80):
    """
    指定された辞書の各DataFrameのヒストグラムをプロットする関数です。

    Args:
        dict_statistics: プロット対象のDataFrameを含む辞書。
        rows: グラフの行数。
        cols: グラフの列数。
        dpi: グラフの解像度。

    Returns:
        None
    """
    plt.subplots_adjust(wspace=0.4, hspace=1.0)
    for year, df in dict_statistics.items():
        print(f"{year}")
        # グラフのサイズを指定
        fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols*4, rows*3), dpi=dpi)

        # 各列のヒストグラムをプロット
        for i, column in enumerate(df.columns):
            ax = axes[i // cols, i % cols]
            df[column].hist(ax=ax, bins=20)  # ヒストグラムのビンの数を調整可能
            ax.set_title(column)

        # グラフのレイアウト調整
        #plt.tight_layout()
        plt.suptitle(f"Histograms for Year {year}", fontsize=14)

        # グラフの保存 (必要に応じて)
        # plt.savefig(f"histograms_{year}.png")
        plt.show()



In [ ]:
# ヒストグラムの作成 (グラフの行数、列数、解像度を適宜変更)
plot_histograms(dict_statistics, rows=4, cols=8, dpi=72)

2000


IndexError: index 2 is out of bounds for axis 0 with size 2

<Figure size 640x480 with 0 Axes>